In [1]:
import numpy as np
from numpy.linalg import norm

In [2]:
RUNS = 25
K = 10

In [3]:
def parseVector(line):
    return np.array([float(x) for x in line.split(' ')])


def closestPoint(p, centers):
    bestI = 0
    closest = float("+inf")
    
    for i in range(len(centers)):
        temp = np.sum((p - centers[i]) ** 2)
        
        if temp < closest:
                closest = temp
                bestI = i
                
    return bestI

In [4]:
import os
import findspark
os.environ["PYSPARK_PYTHON"] = "python3"
findspark.init("spark-2.4.2-bin-hadoop2.7/",)

In [5]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
        .master("local[*]")
        .appName("LectureExample")
        .getOrCreate()
)
sc = spark.sparkContext

In [6]:
lines = sc.textFile('c1.txt')
centroids = lines.map(parseVector)

In [7]:
lines0 = sc.textFile('data.txt')
data0 = lines0.map(parseVector)

In [8]:
data = (data0.map(lambda p: (p, [np.inf] * RUNS)).cache())

In [9]:
centers = centroids

def update_dist(vecIN, distIN, kIN):
    new_dist = norm(vec - centers[:, kIN], axis=1)**2
    return np.min([distIN, new_dist], axis=0)

In [18]:
for idx in range(K-1):
    data = (data.map(lambda p: update_dist(p,dist,idx)))